In [ ]:
import datetime
import numpy as np

In [ ]:
# Setup Project Parameters
DataAugmentation, DataAugThreshold = True, 30000
DataFilePath, DataFileName, FileType = "autodl-tmp/Multitriage/Data/powershell", "Issueazure-powershellWebScrap", ".csv"
MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = 300, 100
LoadDataAugFromFile = False
LearningRate = 0.001
VALIDATION_SPLIT = 0.2

In [ ]:
# log file
filename = 'Multimodel' + '_' + DataFileName + ('_dataaug' if DataAugmentation else '') + '_' + str(LearningRate) + '_' + str(MAX_SEQUENCE_LENGTH) + '_' + str(EMBEDDING_DIM) 
filelog = open(filename + ".txt", "w")
filelog.write("StartTime:" + str(datetime.datetime.now()))
filelog.close()